###In this work, the statistical significance of differences in key metrics between test and control groups was evaluated across four separate A/B tests, and the results were summarized in a structured final table.

###[View Tableau Dashboard](https://public.tableau.com/app/profile/tetiana.kholod/viz/ABtestingtoolCalculator/ABtest)


In [ ]:
#!pip install --upgrade google-cloud-bigquery


In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
from google.colab import auth

import matplotlib.pyplot as plt
from matplotlib import ticker
import numpy as np
import pandas as pd
from scipy.stats import normaltest, shapiro
from scipy.stats import norm

In [ ]:
auth.authenticate_user()

In [ ]:
client = bigquery.Client(project="data-analytics-mate")

In [ ]:
query = """

with session_info as (
  SELECT
    s.date,
    s.ga_session_id,
    sp,country,
    sp.device,
    sp.continent,
    sp.channel,
    ab.test,
    ab.test_group
  FROM `DA.ab_test` as ab
  join `DA.session_params` as sp
  on ab.ga_session_id = sp.ga_session_id
  join `DA.session` as s
  on s.ga_session_id = ab.ga_session_id
),
session_with_order as (
  SELECT
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   count (DISTINCT o.ga_session_id) as session_with_order
  FROM `DA.order` as o
  join session_info
  ON o.ga_session_id = session_info.ga_session_id
  GROUP BY
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group
),
events as(
  SELECT
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   sp.event_name,
   count (sp.ga_session_id) as events_cnt
  FROM `DA.event_params` as sp
  JOIN session_info
  ON session_info.ga_session_id = sp.ga_session_id
  GROUP BY
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   sp.event_name
),
sessions as (
  SELECT
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   count (DISTINCT session_info.ga_session_id) as sessions_cnt
  FROM session_info
  GROUP BY
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group
),
accounts as(
  SELECT
   session_info.date,
   session_info.country,
   session_info.continent,
   session_info.device,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   count (acs.ga_session_id) as new_accounts_cnt
  FROM `DA.account_session` as acs
  JOIN session_info
  ON session_info.ga_session_id = acs.ga_session_id
  GROUP BY
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group
)


SELECT
session_with_order.date,
   session_with_order.continent,
   session_with_order.country,
   session_with_order.device,
   session_with_order.channel,
   session_with_order.test,
   session_with_order.test_group,
   'session_with_order' as event_name,
   session_with_order.session_with_order as value
FROM session_with_order
UNION ALL


SELECT
events.date,
   events.continent,
   events.country,
   events.device,
   events.channel,
   events.test,
   events.test_group,
   events.event_name,
   events.events_cnt as value
FROM events


UNION ALL
SELECT
   accounts.date,
   accounts.continent,
   accounts.country,
   accounts.device,
   accounts.channel,
   accounts.test,
   accounts.test_group,
   'new_accounts' as event_name,
   accounts.new_accounts_cnt as value
FROM accounts


UNION ALL
SELECT
   sessions.date,
   sessions.continent,
   sessions.country,
   sessions.device,
   sessions.channel,
   sessions.test,
   sessions.test_group,
   'sessions' as event,
   sessions_cnt as value
   FROM sessions
"""

In [ ]:
query_job = client.query(query)  # Execution of SQL query
results = query_job.result()  # Waiting for query to complete

In [ ]:
data = results.to_dataframe()

In [ ]:
data.head(1)


,date,continent,country,device,channel,test,test_group,event_name,value
0,2020-11-01,Americas,Jamaica,desktop,Organic Search,2,1,session_with_order,1


#METRICS

add_payment_info / session  
add_shipping_info / session  
begin_checkout / session  
new_accounts / session

In [ ]:
#data["event_name"].unique()

In [ ]:
#List of Events
events_list = list(data["event_name"].unique())
print(events_list)

['session_with_order', 'new_accounts', 'sessions', 'page_view', 'first_visit', 'user_engagement', 'scroll', 'view_promotion', 'view_item', 'begin_checkout', 'add_shipping_info', 'session_start', 'add_to_cart', 'add_payment_info', 'select_promotion', 'select_item', 'view_search_results', 'click', 'view_item_list']


In [ ]:
#to add columns for events (1 / 0)
results = data
events_list = list(data["event_name"].unique())
for event in events_list:
    results[event] = ((results['event_name'] == event).astype(int)) * results['value']
results.head()

,date,continent,country,device,channel,test,test_group,event_name,value,session_with_order,...,begin_checkout,add_shipping_info,session_start,add_to_cart,add_payment_info,select_promotion,select_item,view_search_results,click,view_item_list
0,2020-11-01,Americas,Jamaica,desktop,Organic Search,2,1,session_with_order,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2020-11-02,Europe,North Macedonia,desktop,Organic Search,2,2,session_with_order,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2020-11-03,Africa,Kenya,desktop,Undefined,2,2,session_with_order,1,1,...,0,0,0,0,0,0,0,0,0,0
3,2020-11-04,Americas,Ecuador,desktop,Organic Search,2,1,session_with_order,1,1,...,0,0,0,0,0,0,0,0,0,0
4,2020-11-05,Europe,Bulgaria,desktop,Direct,2,1,session_with_order,1,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Define the total number of events by test and group
results_grouped_by_test_group = results.groupby(['test', 'test_group'])[events_list].sum()
results_grouped_by_test_group.head()

session_with_order  new_accounts  sessions  page_view  \
test test_group                                                          
1    1                         4514          3823     45362     191543   
     2                         4526          3681     45193     198050   
2    1                         5102          4165     50637     220275   
     2                         5003          4184     50244     212320   
3    1                         6951          5856     70047     286351   

                 first_visit  user_engagement  scroll  view_promotion  \
test test_group                                                         
1    1                 30596           171788   73244           29188   
     2                 30512           179081   73376           29117   
2    1                 34511           198266   80713           32367   
     2                 34171           189931   81370           31680   
3    1                 50438           249921  110360           41169   

                 view_item  begin_checkout  add_shipping_info  session_start  \
test test_group                                                                
1    1               62335            3784               3034          45905   
     2               65337            4021               3221          45649   
2    1               72717            4262               3480          51219   
     2               68700            4313               3510          50808   
3    1               93931            9532               5298          71312   

                 add_to_cart  add_payment_info  select_promotion  select_item  \
test test_group                                                                 
1    1                  1395              1988              1275          543   
     2                  1366              2229              1323          530   
2    1                  2811              2344              1477          905   
     2                  3061              2409              1406          946   
3    1                 17674              3623              2020         8735   

                 view_search_results  click  view_item_list  
test test_group                                              
1    1                          3678    368              27  
     2                          3882    353              24  
2    1                          4282    337              24  
     2                          4198    413              29  
3    1                          5764    280               9

In [ ]:
results_grouped_by_test_group = results_grouped_by_test_group.reset_index()
results_grouped_by_test_group['test'] = results_grouped_by_test_group['test'].ffill()
results_grouped_by_test_group.head()

,test,test_group,session_with_order,new_accounts,sessions,page_view,first_visit,user_engagement,scroll,view_promotion,...,begin_checkout,add_shipping_info,session_start,add_to_cart,add_payment_info,select_promotion,select_item,view_search_results,click,view_item_list
0,1,1,4514,3823,45362,191543,30596,171788,73244,29188,...,3784,3034,45905,1395,1988,1275,543,3678,368,27
1,1,2,4526,3681,45193,198050,30512,179081,73376,29117,...,4021,3221,45649,1366,2229,1323,530,3882,353,24
2,2,1,5102,4165,50637,220275,34511,198266,80713,32367,...,4262,3480,51219,2811,2344,1477,905,4282,337,24
3,2,2,5003,4184,50244,212320,34171,189931,81370,31680,...,4313,3510,50808,3061,2409,1406,946,4198,413,29
4,3,1,6951,5856,70047,286351,50438,249921,110360,41169,...,9532,5298,71312,17674,3623,2020,8735,5764,280,9


METRICS  

add_payment_info / session  

add_shipping_info / session  

begin_checkout / session  

new_accounts / session

In [ ]:
test_list = results_grouped_by_test_group["test"].unique()
test_group_list = results_grouped_by_test_group["test_group"].unique()
events_list_project2 = ["add_payment_info", "add_shipping_info", "begin_checkout", "new_accounts"]

final_table = pd.DataFrame()
for test in test_list:

    results_test_group1 = results_grouped_by_test_group[(results_grouped_by_test_group["test"] == test) & (results_grouped_by_test_group["test_group"] == 1)]
    results_test_group2 = results_grouped_by_test_group[(results_grouped_by_test_group["test"] == test) & (results_grouped_by_test_group["test_group"] == 2)]

    for event in events_list_project2:


        # to create one row
        new_row_1 = {
        "test_number": test,
            #"test_group": test_group,
        "metric": event,
        "numerator_control": results_test_group1[event].iloc[0],
        "denominator_control": results_test_group1["sessions"].iloc[0],
        "conversion_control": results_test_group1[event].iloc[0] / results_test_group1["sessions"].iloc[0],
        "numerator_test": results_test_group2[event].iloc[0],
        "denominator_test": results_test_group2["sessions"].iloc[0],
        "conversion_test": results_test_group2[event].iloc[0] / results_test_group2["sessions"].iloc[0],

        "metric_change": (),
        "z_stat": (),
        "p_value": (),
        "significant": (),

        "dimension": "total",
        "dimension_value": "total"
        }

        final_table = pd.concat([final_table, pd.DataFrame([new_row_1])], ignore_index=True)
#final_table["dimension"] = "total"
#final_table["dimension_value"] = "total"

final_table.head(50)




,test_number,metric,numerator_control,denominator_control,conversion_control,numerator_test,denominator_test,conversion_test,metric_change,z_stat,p_value,significant,dimension,dimension_value
0,1,add_payment_info,1988,45362,0.043825,2229,45193,0.049322,(),(),(),(),total,total
1,1,add_shipping_info,3034,45362,0.066884,3221,45193,0.071272,(),(),(),(),total,total
2,1,begin_checkout,3784,45362,0.083418,4021,45193,0.088974,(),(),(),(),total,total
3,1,new_accounts,3823,45362,0.084278,3681,45193,0.081451,(),(),(),(),total,total
4,2,add_payment_info,2344,50637,0.046290,2409,50244,0.047946,(),(),(),(),total,total
5,2,add_shipping_info,3480,50637,0.068724,3510,50244,0.069859,(),(),(),(),total,total
6,2,begin_checkout,4262,50637,0.084168,4313,50244,0.085841,(),(),(),(),total,total
7,2,new_accounts,4165,50637,0.082252,4184,50244,0.083274,(),(),(),(),total,total
8,3,add_payment_info,3623,70047,0.051722,3697,70439,0.052485,(),(),(),(),total,total
9,3,add_shipping_info,5298,70047,0.075635,5188,70439,0.073652,(),(),(),(),total,total


In [ ]:
devices_list = results["device"].unique()
#print(devices_list)
for device in devices_list:
    results_grouped_by_test_group_devices = results[results["device"] == device].groupby(['test', 'test_group'])[events_list].sum()

    results_grouped_by_test_group_devices = results_grouped_by_test_group_devices.reset_index()
    results_grouped_by_test_group_devices['test'] = results_grouped_by_test_group_devices['test'].ffill()

    for test in test_list:

        results_test_devices_group_1 = results_grouped_by_test_group_devices[(results_grouped_by_test_group_devices["test"] == test) & (results_grouped_by_test_group_devices["test_group"] == 1)]
        results_test_devices_group_2 = results_grouped_by_test_group_devices[(results_grouped_by_test_group_devices["test"] == test) & (results_grouped_by_test_group_devices["test_group"] == 2)]

        for event in events_list_project2:


            # to create one row
            new_row_2 = {
            "test_number": test,
            #"test_group": test_group,
            "metric": event,
            "numerator_control": results_test_devices_group_1[event].iloc[0],
            "denominator_control": results_test_devices_group_1["sessions"].iloc[0],
            "conversion_control": results_test_devices_group_1[event].iloc[0] / results_test_devices_group_1["sessions"].iloc[0],
            "numerator_test": results_test_devices_group_2[event].iloc[0],
            "denominator_test": results_test_devices_group_2["sessions"].iloc[0],
            "conversion_test": results_test_devices_group_2[event].iloc[0] / results_test_devices_group_2["sessions"].iloc[0],

            "metric_change": (),
            "z_stat": (),
            "p_value": (),
            "significant": (),

            "dimension": "device",
            "dimension_value": device
            }

            final_table = pd.concat([final_table, pd.DataFrame([new_row_2])], ignore_index=True)



#final_table["metric_change"] = final_table["conversion_test"] / final_table["conversion_control"] - 1
final_table.head(150)






,test_number,metric,numerator_control,denominator_control,conversion_control,numerator_test,denominator_test,conversion_test,metric_change,z_stat,p_value,significant,dimension,dimension_value
0,1,add_payment_info,1988,45362,0.043825,2229,45193,0.049322,(),(),(),(),total,total
1,1,add_shipping_info,3034,45362,0.066884,3221,45193,0.071272,(),(),(),(),total,total
2,1,begin_checkout,3784,45362,0.083418,4021,45193,0.088974,(),(),(),(),total,total
3,1,new_accounts,3823,45362,0.084278,3681,45193,0.081451,(),(),(),(),total,total
4,2,add_payment_info,2344,50637,0.046290,2409,50244,0.047946,(),(),(),(),total,total
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,3,new_accounts,126,1537,0.081978,137,1573,0.087095,(),(),(),(),device,tablet
60,4,add_payment_info,107,2379,0.044977,54,2360,0.022881,(),(),(),(),device,tablet
61,4,add_shipping_info,125,2379,0.052543,88,2360,0.037288,(),(),(),(),device,tablet
62,4,begin_checkout,311,2379,0.130727,232,2360,0.098305,(),(),(),(),device,tablet


In [ ]:
channel_list = results["channel"].unique()
#print(devices_list)
for channel in channel_list:
    results_grouped_by_test_group_channels = results[results["channel"] == channel].groupby(['test', 'test_group'])[events_list].sum()

    results_grouped_by_test_group_channels = results_grouped_by_test_group_channels.reset_index()
    results_grouped_by_test_group_channels['test'] = results_grouped_by_test_group_channels['test'].ffill()

    for test in test_list:

        results_test_channels_group_1 = results_grouped_by_test_group_channels[(results_grouped_by_test_group_channels["test"] == test) & (results_grouped_by_test_group_channels["test_group"] == 1)]
        results_test_channels_group_2 = results_grouped_by_test_group_channels[(results_grouped_by_test_group_channels["test"] == test) & (results_grouped_by_test_group_channels["test_group"] == 2)]

        for event in events_list_project2:


            # to create one row
            new_row_3 = {
            "test_number": test,
            #"test_group": test_group,
            "metric": event,
            "numerator_control": results_test_channels_group_1[event].iloc[0],
            "denominator_control": results_test_channels_group_1["sessions"].iloc[0],
            "conversion_control": results_test_channels_group_1[event].iloc[0] / results_test_channels_group_1["sessions"].iloc[0],
            "numerator_test": results_test_channels_group_2[event].iloc[0],
            "denominator_test": results_test_channels_group_2["sessions"].iloc[0],
            "conversion_test": results_test_channels_group_2[event].iloc[0] / results_test_channels_group_2["sessions"].iloc[0],

            "metric_change": (),
            "z_stat": (),
            "p_value": (),
            "significant": (),

            "dimension": "channel",
            "dimension_value": channel
            }

            final_table = pd.concat([final_table, pd.DataFrame([new_row_3])], ignore_index=True)

final_table["metric_change"] = final_table["conversion_test"] / final_table["conversion_control"] - 1
final_table.head(150)

,test_number,metric,numerator_control,denominator_control,conversion_control,numerator_test,denominator_test,conversion_test,metric_change,z_stat,p_value,significant,dimension,dimension_value
0,1,add_payment_info,1988,45362,0.043825,2229,45193,0.049322,0.125420,(),(),(),total,total
1,1,add_shipping_info,3034,45362,0.066884,3221,45193,0.071272,0.065605,(),(),(),total,total
2,1,begin_checkout,3784,45362,0.083418,4021,45193,0.088974,0.066606,(),(),(),total,total
3,1,new_accounts,3823,45362,0.084278,3681,45193,0.081451,-0.033543,(),(),(),total,total
4,2,add_payment_info,2344,50637,0.046290,2409,50244,0.047946,0.035769,(),(),(),total,total
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,3,new_accounts,474,5726,0.082780,463,5780,0.080104,-0.032333,(),(),(),channel,Social Search
140,4,add_payment_info,474,7961,0.059540,380,8056,0.047170,-0.207766,(),(),(),channel,Social Search
141,4,add_shipping_info,587,7961,0.073734,530,8056,0.065789,-0.107751,(),(),(),channel,Social Search
142,4,begin_checkout,1438,7961,0.180631,1318,8056,0.163605,-0.094258,(),(),(),channel,Social Search


In [ ]:
country_list = results["country"].unique()
#print(devices_list)
for country in country_list:
    results_grouped_by_test_group_countries = results[results["country"] == channel].groupby(['test', 'test_group'])[events_list].sum()

    results_grouped_by_test_group_countries = results_grouped_by_test_group_countries.reset_index()
    results_grouped_by_test_group_countries['test'] = results_grouped_by_test_group_countries['test'].ffill()

    for test in test_list:

        results_test_countries_group_1 = results_grouped_by_test_group_countries[(results_grouped_by_test_group_countries["test"] == test) & (results_grouped_by_test_group_countries["test_group"] == 1)]
        results_test_countries_group_2 = results_grouped_by_test_group_countries[(results_grouped_by_test_group_countries["test"] == test) & (results_grouped_by_test_group_countries["test_group"] == 2)]

        for event in events_list_project2:


            # to create one row
            new_row_5 = {
            "test_number": test,
            #"test_group": test_group,
            "metric": event,
            "numerator_control": results_test_channels_group_1[event].iloc[0],
            "denominator_control": results_test_channels_group_1["sessions"].iloc[0],
            "conversion_control": results_test_channels_group_1[event].iloc[0] / results_test_channels_group_1["sessions"].iloc[0],
            "numerator_test": results_test_channels_group_2[event].iloc[0],
            "denominator_test": results_test_channels_group_2["sessions"].iloc[0],
            "conversion_test": results_test_channels_group_2[event].iloc[0] / results_test_channels_group_2["sessions"].iloc[0],

            "metric_change": (),
            "z_stat": (),
            "p_value": (),
            "significant": (),

            "dimension": "channel",
            "dimension_value": channel
            }

            final_table = pd.concat([final_table, pd.DataFrame([new_row_5])], ignore_index=True)

final_table["metric_change"] = final_table["conversion_test"] / final_table["conversion_control"] - 1
final_table.head(150)

,test_number,metric,numerator_control,denominator_control,conversion_control,numerator_test,denominator_test,conversion_test,metric_change,z_stat,p_value,significant,dimension,dimension_value
0,1,add_payment_info,1988,45362,0.043825,2229,45193,0.049322,0.125420,(),(),(),total,total
1,1,add_shipping_info,3034,45362,0.066884,3221,45193,0.071272,0.065605,(),(),(),total,total
2,1,begin_checkout,3784,45362,0.083418,4021,45193,0.088974,0.066606,(),(),(),total,total
3,1,new_accounts,3823,45362,0.084278,3681,45193,0.081451,-0.033543,(),(),(),total,total
4,2,add_payment_info,2344,50637,0.046290,2409,50244,0.047946,0.035769,(),(),(),total,total
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,1,add_shipping_info,587,7961,0.073734,530,8056,0.065789,-0.107751,(),(),(),channel,Social Search
146,1,begin_checkout,1438,7961,0.180631,1318,8056,0.163605,-0.094258,(),(),(),channel,Social Search
147,1,new_accounts,667,7961,0.083783,653,8056,0.081058,-0.032534,(),(),(),channel,Social Search
148,2,add_payment_info,474,7961,0.059540,380,8056,0.047170,-0.207766,(),(),(),channel,Social Search


In [ ]:

# conversion rate
p1 = final_table["numerator_control"] / final_table["denominator_control"]
p2 = final_table["numerator_test"] / final_table["denominator_test"]

# pooled proportion
p_pool = (final_table["numerator_control"] + final_table["numerator_test"]) / (
    final_table["denominator_control"] + final_table["denominator_test"]
)

# standard error
se = np.sqrt(p_pool * (1 - p_pool) * (1 / final_table["denominator_control"] + 1 / final_table["denominator_test"]))

# z-stat
final_table["z_stat"] = (p2 - p1) / se

# p-value
final_table["p_value"] = 2 * (1 - norm.cdf(np.abs(final_table["z_stat"])))

# signidicance
final_table["significant"] = final_table["p_value"] < 0.05

final_table.head(250)

,test_number,metric,numerator_control,denominator_control,conversion_control,numerator_test,denominator_test,conversion_test,metric_change,z_stat,p_value,significant,dimension,dimension_value
0,1,add_payment_info,1988,45362,0.043825,2229,45193,0.049322,0.125420,3.924884,0.000087,True,total,total
1,1,add_shipping_info,3034,45362,0.066884,3221,45193,0.071272,0.065605,2.603571,0.009226,True,total,total
2,1,begin_checkout,3784,45362,0.083418,4021,45193,0.088974,0.066606,2.978783,0.002894,True,total,total
3,1,new_accounts,3823,45362,0.084278,3681,45193,0.081451,-0.033543,-1.542883,0.122859,False,total,total
4,2,add_payment_info,2344,50637,0.046290,2409,50244,0.047946,0.035769,1.240994,0.214608,False,total,total
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,2,add_shipping_info,587,7961,0.073734,530,8056,0.065789,-0.107751,-1.973819,0.048402,True,channel,Social Search
246,2,begin_checkout,1438,7961,0.180631,1318,8056,0.163605,-0.094258,-2.854395,0.004312,True,channel,Social Search
247,2,new_accounts,667,7961,0.083783,653,8056,0.081058,-0.032534,-0.627241,0.530501,False,channel,Social Search
248,3,add_payment_info,474,7961,0.059540,380,8056,0.047170,-0.207766,-3.484160,0.000494,True,channel,Social Search


In [ ]:
#final_table.info()

In [ ]:
final_table.to_csv('temp2.csv', index=False)
!ls /content


sample_data  temp2.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data.to_csv('/content/drive/MyDrive/temp2.csv', index=False)

Mounted at /content/drive
